In [1]:
import pandas as pd
import numpy as np
import os

### Eerst gaan we alle datafiles die we nodig hebben mergen tot 1 dataframe

#### 1. Alles van Account mergen

##### Account mergen met activiteit tabellen

In [5]:
data_acc = pd.read_csv('../data_clean/Account_fixed.csv', sep=',')
data_acti = pd.read_csv('../data_clean/Activiteitscode_fixed.csv', sep=",")
data_acc_acti = pd.read_csv('../data_clean/Account_activiteitscode_fixed.csv', sep=",")

# merge account_activiteitscode with activiteitscode
merged_relation = data_acc_acti.merge(data_acti, left_on='account_activiteitscode_activiteitscode', right_on='activiteitscode_activiteitscode_id', how='inner')
# merge account with the merged relation from above
merged_accounts = data_acc.merge(merged_relation, left_on='account_account_id', right_on='account_activiteitscode_account', how='inner')

In [6]:
# RAM vrijmaken
try:
    del data_acc
    del data_acti
    del data_acc_acti
    del merged_relation
except:
    print("Data is niet meer beschikbaar / is al verwijderd")

In [7]:
merged_accounts.shape

(14673, 22)

##### Accounts mergen met financiële data

In [8]:
financiele_data = pd.read_csv('../data_clean/Account_financiële_data_fixed.csv', sep=',')
financiele_data.shape

(2030180, 6)

In [9]:
merged_accounts = merged_accounts.merge(financiele_data, left_on='account_account_id', right_on='financieledata_ondernemingid', how='inner')
merged_accounts.shape

(181428, 28)

In [10]:
try:
    del financiele_data
except:
    print("Data is niet meer beschikbaar / is al verwijderd")

##### Accounts merge met Contactfiches

In [11]:
contactfiches = pd.read_csv('../data_clean/Contact_fixed.csv', sep=',')
contactfiches.shape

(1169683, 6)

In [12]:
merged_accounts = merged_accounts.merge(contactfiches, left_on='account_account_id', right_on='contact_account', how='inner')
merged_accounts.shape

(3748445, 34)

In [13]:
try:
    del contactfiches
except:
    print("Data is niet meer beschikbaar / is al verwijderd")

##### Accounts mergen met Functie

In [14]:
contactfiches_functies = pd.read_csv('../data_clean/Contact_functie_fixed.csv', sep=',')
functies = pd.read_csv('../data_clean/Functie_fixed.csv', sep=',')
contactfiches_functies.shape, functies.shape

((589129, 2), (71, 2))

In [15]:
contactfiches_functies.columns

Index(['contactfunctie_contactpersoon', 'contactfunctie_functie'], dtype='object')

In [16]:
merged_accounts = merged_accounts.merge(contactfiches_functies, left_on='contact_contactpersoon_id', right_on='contactfunctie_contactpersoon', how='inner')

In [17]:
# list(merged_accounts.columns)

In [18]:
merged_accounts = merged_accounts.merge(functies, left_on='contactfunctie_functie', right_on='functie_functie_id', how='inner')

In [19]:
merged_accounts.shape

(1771670, 38)

In [20]:
try:
    del contactfiches_functies
    del functies
except:
    print("Data is niet meer beschikbaar / is al verwijderd")

##### Accounts merge met Personen

In [21]:
personen = pd.read_csv('../data_clean/Persoon_fixed.csv', sep=',')
personen.shape

(580981, 29)

In [22]:
merged_accounts = merged_accounts.merge(personen, left_on='contact_persoon_id', right_on='persoon_persoon_id', how='inner')
merged_accounts.shape

(1771670, 67)

In [23]:
try:
    del personen
except:
    print("Data is niet meer beschikbaar / is al verwijderd")

##### Accounts merged tijdelijk opslaan

In [24]:
if not os.path.exists('../data_merged'):
    os.makedirs('../data_merged')

if not os.path.exists('../data_merged/merged_accounts.csv'):
    merged_accounts.to_csv('../data_merged/merged_accounts.csv', sep=',', index=False)
else:
    # remove file
    os.remove('../data_merged/merged_accounts.csv')
    # save file
    merged_accounts.to_csv('../data_merged/merged_accounts.csv', sep=',', index=False)

In [25]:
try:
    del merged_accounts
except:
    print("Data is niet meer beschikbaar / is al verwijderd")

#### 2. Alles van Afspraken mergen

In [26]:
afspraak_betreft_account_cleaned = pd.read_csv('../data_clean/Afspraak_betreft_account_cleaned_fixed.csv', sep=',')
afspraak_account_gelinkt_cleaned = pd.read_csv('../data_clean/Afspraak_account_gelinkt_cleaned_fixed.csv', sep=',')
afspraak_betreft_contact_cleaned = pd.read_csv('../data_clean/Afspraak_betreft_contact_cleaned_fixed.csv', sep=',')

In [27]:
afspraak_betreft_account_cleaned.shape, afspraak_account_gelinkt_cleaned.shape, afspraak_betreft_contact_cleaned.shape

((4876, 7), (2934, 7), (2552, 7))

In [28]:
afspraak_betreft_account_cleaned = afspraak_betreft_account_cleaned.drop(['afspraak_betreft_account_betreft_id'], axis=1)
afspraak_account_gelinkt_cleaned = afspraak_account_gelinkt_cleaned.drop(['afspraak_account_gelinkt_account'], axis=1)
afspraak_betreft_contact_cleaned = afspraak_betreft_contact_cleaned.drop(['afspraak_betreft_contactfiche_betreft_id'], axis=1)

In [29]:
# list(afspraak_betreft_account_cleaned.columns)

In [30]:
# list(afspraak_account_gelinkt_cleaned.columns)

In [31]:
# list(afspraak_betreft_contact_cleaned.columns)

In [32]:
afspraak_betreft_account_cleaned.columns = ['afspraak_afspraak_id', 'afspraak_thema', 'afspraak_subthema', 'afspraak_onderwerp', 'afspraak_eindtijd', 'afspraak_keyphrases']
afspraak_account_gelinkt_cleaned.columns = ['afspraak_afspraak_id', 'afspraak_thema', 'afspraak_subthema', 'afspraak_onderwerp', 'afspraak_eindtijd', 'afspraak_keyphrases']
afspraak_betreft_contact_cleaned.columns = ['afspraak_afspraak_id', 'afspraak_thema', 'afspraak_subthema', 'afspraak_onderwerp', 'afspraak_eindtijd', 'afspraak_keyphrases']

In [33]:
afspraak_betreft_account_cleaned["afspraak_betreft"] = "account"
afspraak_account_gelinkt_cleaned["afspraak_betreft"] = "account"
afspraak_betreft_contact_cleaned["afspraak_betreft"] = "contact"

In [34]:
afspraken_merged = pd.concat([afspraak_betreft_account_cleaned, afspraak_account_gelinkt_cleaned, afspraak_betreft_contact_cleaned])

In [35]:
afspraken = pd.read_csv('../data_clean/Afspraak_alle_fixed.csv', sep=',')
afspraken.columns = ['afspraak_afspraak_id']
afspraken.head()

,afspraak_afspraak_id
0,00082DAA-24BF-ED11-83FF-6045BD8952CE
1,00109C77-4D09-EB11-8114-001DD8B72B61
2,0011A5D9-CC9A-EC11-B400-0022488005A7
3,0016532E-14E3-EC11-BB3C-000D3ABCA38B
4,003760C1-13B4-EC11-983F-00224883C880


In [36]:
afspraken_merged = pd.merge(afspraken, afspraken_merged, on='afspraak_afspraak_id', how='right')
afspraken_merged.reset_index(drop=True, inplace=True)

In [37]:
afspraken_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10362 entries, 0 to 10361
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   afspraak_afspraak_id  10362 non-null  object
 1   afspraak_thema        10362 non-null  object
 2   afspraak_subthema     10362 non-null  object
 3   afspraak_onderwerp    10362 non-null  object
 4   afspraak_eindtijd     10362 non-null  object
 5   afspraak_keyphrases   10362 non-null  object
 6   afspraak_betreft      10362 non-null  object
dtypes: object(7)
memory usage: 566.8+ KB


In [38]:
# RAM vrijmaken
try:
    del afspraak_betreft_account_cleaned
    del afspraak_account_gelinkt_cleaned
    del afspraak_betreft_contact_cleaned
    del afspraken
except:
    print("Data is niet meer beschikbaar / is al verwijderd")

In [39]:
afspraken_merged = afspraken_merged.drop_duplicates(subset=['afspraak_afspraak_id'], keep='first')
afspraken_merged.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7624 entries, 0 to 10357
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   afspraak_afspraak_id  7624 non-null   object
 1   afspraak_thema        7624 non-null   object
 2   afspraak_subthema     7624 non-null   object
 3   afspraak_onderwerp    7624 non-null   object
 4   afspraak_eindtijd     7624 non-null   object
 5   afspraak_keyphrases   7624 non-null   object
 6   afspraak_betreft      7624 non-null   object
dtypes: object(7)
memory usage: 476.5+ KB


In [40]:
activiteit_vereist_contact = pd.read_csv('../data_clean/Activiteit_vereist_contact_fixed.csv', sep=',')
activiteit_vereist_contact.shape

(4635, 2)

In [41]:
afspraken_merged = afspraken_merged.merge(activiteit_vereist_contact, left_on='afspraak_afspraak_id', right_on='activiteitvereistcontact_activityid_id', how='left')
afspraken_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7754 entries, 0 to 7753
Data columns (total 9 columns):
 #   Column                                  Non-Null Count  Dtype 
---  ------                                  --------------  ----- 
 0   afspraak_afspraak_id                    7754 non-null   object
 1   afspraak_thema                          7754 non-null   object
 2   afspraak_subthema                       7754 non-null   object
 3   afspraak_onderwerp                      7754 non-null   object
 4   afspraak_eindtijd                       7754 non-null   object
 5   afspraak_keyphrases                     7754 non-null   object
 6   afspraak_betreft                        7754 non-null   object
 7   activiteitvereistcontact_activityid_id  4545 non-null   object
 8   activiteitvereistcontact_reqattendee    4545 non-null   object
dtypes: object(9)
memory usage: 545.3+ KB


In [42]:
try:
    del activiteit_vereist_contact
except:
    print("Data is niet meer beschikbaar / is al verwijderd")

##### Afspraken tijdelijk opslaan

In [43]:
if not os.path.exists('../data_merged/merged_afspraken.csv'):
    afspraken_merged.to_csv('../data_merged/merged_afspraken.csv', sep=',', index=False)
else:
    # remove file
    os.remove('../data_merged/merged_afspraken.csv')
    # save file
    afspraken_merged.to_csv('../data_merged/merged_afspraken.csv', sep=',', index=False)

In [44]:
try:
    del afspraken_merged
except:
    print("Data is niet meer beschikbaar / is al verwijderd")

#### 3. Alles van Campagnes mergen --> ook CDI_Pageview en CDI_Visits

##### Campagne mergen met Sessie

In [5]:
campagne = pd.read_csv('../data_clean/Campagne_fixed.csv', sep=',')
campagne.shape

(4092, 11)

In [6]:
sessie = pd.read_csv('../data_clean/Sessie_fixed.csv', sep=',')
sessie.shape

(9403, 8)

In [7]:
campagnes_merged = campagne.merge(sessie, left_on='campagne_campagne_id', right_on='sessie_campagne', how='inner')
campagnes_merged.shape

(9403, 19)

In [8]:
try:
    del campagne
    del sessie
except:
    print("Data is niet meer beschikbaar / is al verwijderd")

##### Campagne mergen met Inschrijving

In [9]:
inschrijving = pd.read_csv('../data_clean/Inschrijving_fixed.csv', sep=',')
inschrijving.shape

(49175, 8)

In [10]:
campagnes_merged = campagnes_merged.merge(inschrijving, left_on='campagne_campagne_id', right_on='inschrijving_campagne', how='inner')
campagnes_merged.shape

(112742, 27)

In [11]:
try:
    del inschrijving
except:
    print("Data is niet meer beschikbaar / is al verwijderd")

##### Campagnes mergen met Sessie inschrijving

In [12]:
sessie_inschrijving = pd.read_csv('../data_clean/Sessie_inschrijving_fixed.csv', sep=',')
sessie_inschrijving.shape

(88309, 3)

In [13]:
campagnes_merged = campagnes_merged.merge(sessie_inschrijving, left_on='sessie_sessie_id', right_on='sessieinschrijving_sessie', how='inner')
campagnes_merged.shape

(8139508, 30)

In [14]:
try:
    del sessie_inschrijving
except:
    print("Data is niet meer beschikbaar / is al verwijderd")

In [55]:
# campagnes_merged.columns # campagne_campagne_id

In [15]:
try:
    campagnes_merged.drop(['inschrijving_campagne', 'inschrijving_campagne_naam_', 'sessie_campagne', 'sessieinschrijving_sessie', 'sessieinschrijving_inschrijving', 'sessieinschrijving_sessieinschrijving_id'], axis=1, inplace=True)
    campagnes_merged.drop_duplicates(inplace=True)
    campagnes_merged.shape
except:
    print("Kolommen zijn al gedropped")

In [16]:
campagnes_merged.shape

(111863, 24)

##### Pageviews and Visits mergen om daarna met campagne_merged te mergen

In [17]:
pageview = pd.read_csv('../data_clean/CDI_pageviews_fixed.csv', sep=',')
pageview.shape

(1651, 20)

In [61]:
# list(pageview.columns) # visit

In [18]:
visits = pd.read_csv('../data_clean/CDI_visits_fixed.csv', sep=',')
visits.drop(['visit_campagne_code'], axis=1, inplace=True)
visits.shape

(24444, 34)

In [63]:
# list(visits.columns) # visit_visit_id

In [19]:
visits_pageviews_merged = visits.merge(pageview, left_on='visit_visit_id', right_on='visit', how='left')
visits_pageviews_merged.drop_duplicates(inplace=True)
visits_pageviews_merged.shape

(25187, 54)

In [20]:
try:
    del pageview
    del visits
except:
    print("Data is niet meer beschikbaar / is al verwijderd")

In [21]:
visits_pageviews_merged.drop(['visit', 'visit_campaign', 'visit_contact'], axis=1, inplace=True)

In [22]:
visits_pageviews_merged.shape

(25187, 51)

##### Visit_pageviews mergen met mailing_merged

In [23]:
mailing = pd.read_csv('../data_clean/CDI_mailing_fixed.csv', sep=',')
mailing.drop(['mailing_sent_on'], axis=1, inplace=True)
mailing.shape

(860, 3)

In [24]:
mailing.head()

,mailing_mailing_id,mailing_name,mailing_onderwerp
0,00096FAA-9985-EC11-8D21-6045BD8F7A7F,OV-GR-Trajecten familiebedrijven,Familienieuws: groeiverhalen en -trajecten voo...
1,0087CD38-0CEB-ED11-8849-6045BD895CDC,[CLONE]: OV-MATCH-Uitnodiging Welkom@Voka-Juni...,[CLONE]: Uitnodiging: Welkom@Voka - het welkom...
2,00A0AA7D-F9E4-ED11-A7C7-6045BD8952D3,OV-NW-Voka Politica Zottegem - 24 mei 2023,Uitnodiging: Voka Politica in Zottegem 24 mei
3,00B39325-A66C-EC11-8943-000D3A279FAC,OV-IN-Change management en AI opleiding,Opleiding Change Management: neem jij ook deel?
4,00B816C5-A56C-ED11-9561-6045BD895CDC,OV-NW-Nieuwjaarsreceptie 2023 Gent,Nieuwjaarsreceptie Gent | Voka Oost-Vlaanderen


In [25]:
sentmailclick = pd.read_csv('../data_clean/CDI_sent_email_clicks_fixed.csv', sep=',')
sentmailclick.drop(['sentemail_kliks_contact', 'sentemail_kliks_sent_email_id'], axis=1, inplace=True)
sentmailclick.shape

(57774, 2)

In [26]:
sentmailclick.head()

,sentemail_kliks_clicks,sentemail_kliks_e_mail_versturen
0,1,034044AC-B698-EC11-B400-002248800905
1,1,7C30BC82-233E-EC11-8125-001DD8B72B61
2,1,10B4EAC6-C9E4-ED11-A7C7-6045BD895D0D
3,1,39F68F4F-4480-EC11-8D21-6045BD8E028B
4,1,5BBCF41D-16F5-EB11-8125-001DD8B72B62


In [27]:
mailing_merged = mailing.merge(sentmailclick, left_on='mailing_mailing_id', right_on='sentemail_kliks_e_mail_versturen', how='inner')
mailing_merged.drop(['sentemail_kliks_e_mail_versturen'], axis=1, inplace=True)

In [28]:
mailing_merged.shape

(57774, 4)

In [29]:
mailing_merged.columns

Index(['mailing_mailing_id', 'mailing_name', 'mailing_onderwerp',
       'sentemail_kliks_clicks'],
      dtype='object')

In [30]:
visits_pageviews_mailing_merged = visits_pageviews_merged.merge(mailing_merged, left_on='visit_email_send', right_on='mailing_mailing_id', how='left')
visits_pageviews_mailing_merged.shape

(5964813, 55)

In [31]:
try:
    del mailing
    del sentmailclick
    del mailing_merged
    del visits_pageviews_merged
except:
    print("Data is niet meer beschikbaar / is al verwijderd")

In [32]:
visits_pageviews_mailing_merged.drop([
    'webcontent', 'visitorkey', 'visit_visit_id', 'aangemaaktop',
    'pageview_id', 'viewedon', 'visit_adobe_reader',
    'visit_browser', 'visit_contact_naam_', 'visit_containssocialprofile',
    'visit_email_send', 'visit_ended_on', 'visit_entry_page', 'visit_exit_page',
    'visit_ip_address', 'visit_ip_organization', 'visit_keywords',
    'visit_ip_longitude', 'visit_ip_latitude', 'visit_operating_system',
    'visit_started_on', 'visit_time', 'visit_referrer', 'visit_referrer_type',
    'visit_score', 'visit_total_pages', 'visit_ip_status', 'visit_aangemaakt_op',
    'visit_gewijzigd_op'
], axis=1, inplace=True)

In [33]:
print(visits_pageviews_mailing_merged.shape)
visits_pageviews_mailing_merged.drop_duplicates(inplace=True)
visits_pageviews_mailing_merged.shape

(5964813, 26)


(280060, 26)

In [ ]:
sorted(list(visits_pageviews_mailing_merged.columns))

##### Campagne_merged mergen met visits_pageviews_merged

In [34]:
campagnes_merged = campagnes_merged.merge(visits_pageviews_mailing_merged, left_on='campagne_campagne_id', right_on='campaign', how='inner')
campagnes_merged.drop_duplicates(inplace=True)
campagnes_merged.shape

(8700097, 50)

In [35]:
campagnes_merged.drop([
    'campagne_campagne_id',
    'campagne_campagne_nr',
    'campagne_einddatum',
    'campagne_startdatum',
    'campaign',
    'contact',
    'gewijzigddoor',
    'gewijzigdop',
    'inschrijving_datum_inschrijving',
    'inschrijving_inschrijving_id',
    'mailing_mailing_id',
    'sessie_sessie_id',
    'sessie_sessie_nr_',
    'sessie_start_datum_tijd',
    
], axis=1, inplace=True)

In [36]:
campagnes_merged.drop_duplicates(inplace=True)
campagnes_merged.shape

(7343170, 36)

##### Campagnes_merged opslaan

In [37]:
if not os.path.exists('../data_merged/merged_campagnes.csv'):
    campagnes_merged.to_csv('../data_merged/merged_campagnes.csv', sep=',', index=False)
else:
    # remove file
    os.remove('../data_merged/merged_campagnes.csv')
    # save file
    campagnes_merged.to_csv('../data_merged/merged_campagnes.csv', sep=',', index=False)

In [38]:
try:
    del campagnes_merged
except:
    print("Data is niet meer beschikbaar / is al verwijderd")

#### 4. Alles mergen tot 1 dataframe

In [39]:
accounts_merged = pd.read_csv('../data_merged/merged_accounts_cleaned.csv', sep=',')
accounts_merged.shape

(26796, 25)

In [73]:
# accounts_merged.columns # contact_contactpersoon_id

In [40]:
campagnes_merged = pd.read_csv('../data_merged/merged_campagnes_cleaned.csv', sep=',')
campagnes_merged.shape

C:\Users\buyse\AppData\Local\Temp\ipykernel_2472\2472515788.py:1: DtypeWarning: Columns (9,16) have mixed types. Specify dtype option on import or set low_memory=False.
  campagnes_merged = pd.read_csv('../data_merged/merged_campagnes_cleaned.csv', sep=',')


(226866, 22)

In [75]:
# campagnes_merged.columns # inschrijving_contactfiche

##### Merge account_merged en campagnes_merged

In [41]:
acc_camp_merged = accounts_merged.merge(campagnes_merged, left_on='contact_contactpersoon_id', right_on='inschrijving_contactfiche', how='inner')
acc_camp_merged.shape

(450090, 47)

In [43]:
acc_camp_merged.drop(['inschrijving_contactfiche'], axis=1, inplace=True)

##### Merge acc_camp_merged met afspraken_merged

In [44]:
afspraken_merged = pd.read_csv('../data_merged/merged_afspraken.csv', sep=',')
afspraken_merged.shape

(7754, 9)

In [45]:
afspraken_merged.columns

Index(['afspraak_afspraak_id', 'afspraak_thema', 'afspraak_subthema',
       'afspraak_onderwerp', 'afspraak_eindtijd', 'afspraak_keyphrases',
       'afspraak_betreft', 'activiteitvereistcontact_activityid_id',
       'activiteitvereistcontact_reqattendee'],
      dtype='object')

In [46]:
merged_total = acc_camp_merged.merge(afspraken_merged, left_on='contact_contactpersoon_id', right_on='activiteitvereistcontact_reqattendee', how='inner')

In [47]:
merged_total.shape

(380391, 55)

##### Merged_total opslaan -- duurt +- 2,5 minuten

In [48]:
merged_total.to_csv('../data_merged/merged_total.csv', sep=',', index=False)

In [49]:
try:
    del accounts_merged
    del campagnes_merged
    del acc_camp_merged
    del afspraken_merged
    
except:
    print("Data is niet meer beschikbaar / is al verwijderd")